In [1]:
!date

Mon Oct 30 08:53:16 UTC 2023


In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from deepface import DeepFace
from pathlib import Path
import cv2
import os
from azure.identity import ManagedIdentityCredential
from dl_conn import get_dl_conn
from dl_orm import DLorm

2023-11-01 12:22:59.788680: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


If there is the impression that there are missing selfies, i.e. there isn't three selfies per user available, you can run this notebook the check how many selfies are missing and of whom.
To fix this, you can run the script in '../../src/data/missing_selfies.py'

In [3]:
# we run over all users in the selfies_dir and check whether they have less than two pictures, if so we add them to the list including the amount of selfies
def list_files_with_extension(directory, extension):
    files = []
    for filename in os.listdir(directory):
        if filename.endswith(extension):
            files.append(os.path.join(directory, filename))
    return files

selfies_dir = Path('../../src/data/selfies')
users_selfie_paths = []
for user in selfies_dir.glob('*'):
    selfies = [user] + list_files_with_extension(user, '.jpg')
    users_selfie_paths.append(selfies)

In [4]:
df_selfiepaths = pd.DataFrame(users_selfie_paths, columns=['user_id', 'selfie1', 'selfie2', 'selfie3'])
df_selfiepaths['user_id'] = df_selfiepaths['user_id'].apply(lambda x : os.path.basename(x)).astype(int)
df_selfiepaths['selfie_link_id_1'] = df_selfiepaths['selfie1'].apply(lambda x : os.path.basename(x).split('_',1)[1].split('.')[0] if isinstance(x, str) else None)
df_selfiepaths['selfie_link_id_2'] = df_selfiepaths['selfie2'].apply(lambda x : os.path.basename(x).split('_',1)[1].split('.')[0] if isinstance(x, str) else None)
df_selfiepaths['selfie_link_id_3'] = df_selfiepaths['selfie3'].apply(lambda x : os.path.basename(x).split('_',1)[1].split('.')[0] if isinstance(x, str) else None)
df_selfiepaths['missing_count'] = df_selfiepaths[['selfie1', 'selfie2', 'selfie3']].isnull().sum(axis=1)
users_missing_selfies = df_selfiepaths[df_selfiepaths['missing_count']>0]
users_missing_selfies.head()

,user_id,selfie1,selfie2,selfie3,selfie_link_id_1,selfie_link_id_2,selfie_link_id_3,missing_count


In [9]:
# number of users missing selfies, missing exactly three/two/one selfie
[users_missing_selfies.user_id.nunique(), 
users_missing_selfies[users_missing_selfies.missing_count == 3].user_id.nunique(), 
users_missing_selfies[users_missing_selfies.missing_count == 2].user_id.nunique(), 
users_missing_selfies[users_missing_selfies.missing_count == 1].user_id.nunique()]

[0, 0, 0, 0]